In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

In [2]:
full_train_orig = pd.read_csv("../Data/application_train.csv")
test =  pd.read_csv("../Data/application_test.csv")
bureau = pd.read_csv("../Data/bureau.csv")

In [3]:
'''
Removing columns with more than 100 null values (filters out 62 columns out of 122 columns)
And not useful (based on EDA) which removes reduces feature size to 43)
'''

null_columns = full_train_orig.columns[full_train_orig.isnull().sum().values > 100].values.tolist()
useless_columns = ["FLAG_DOCUMENT_2", "FLAG_DOCUMENT_4", "FLAG_DOCUMENT_5", "FLAG_DOCUMENT_7"
                  ,'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']

In [4]:
### Manually removing columns which doesn't make sense (based on EDA)
full_train = full_train_orig.drop(null_columns+useless_columns, axis = 1)

In [8]:
#cat_columns_all = set(full_train.columns) - set(full_train._get_numeric_data().columns.tolist())

In [9]:
# cat_feats = ["CODE_GENDER", "NAME_CONTRACT_TYPE", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", 
#              "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "OCCUPATION_TYPE",
#              "NAME_HOUSING_TYPE", "FLAG_DOCUMENT_3"]
# num_feats = ["AMT_INCOME_TOTAL", "CNT_CHILDREN", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "DAYS_BIRTH",
#              "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_LAST_PHONE_CHANGE", "AMT_REQ_CREDIT_BUREAU_YEAR", 
#             "REGION_POPULATION_RELATIVE", ]

In [10]:
full_train_y = full_train.TARGET.values
full_train = full_train.drop(["TARGET"], axis = 1)
full_train = full_train.set_index("SK_ID_CURR")
num_feats = full_train._get_numeric_data().columns.values.tolist()
cat_feats = list(set(full_train.columns.values) - set(num_feats))

In [11]:
## Categorical Features - Train
train_cat= full_train[cat_feats]
train_cat = pd.get_dummies(train_cat)

## Numerical Features - Train
train_num = full_train[num_feats]
# train[num_feats].isnull().any() # Checking if the column has any null value
#train_num = train_num.fillna((train_num.mean()), inplace=True)

## Categorical Features - Test
test_cat = test[cat_feats]
test_cat = pd.get_dummies(test_cat)

## Numerical Features - Test
test_num = test[num_feats]
test_num = test_num.fillna(test_num.mean())

# ## Standardize numerical features
# std = StandardScaler().fit(train_num)
# train_num = std.transform(train_num)
# test_num = std.transform(test_num)


In [12]:
full_train_feats = pd.concat([train_num, train_cat], axis=1)
test_feats = pd.concat([test_num, test_cat], axis=1)

In [13]:
full_train_feats = full_train_feats.apply(lambda x: x.fillna(x.mean()),axis=0)
test_feats = test_feats.apply(lambda x: x.fillna(x.mean()),axis=0)

In [14]:
train_X, valid_X, train_y, valid_y = train_test_split(full_train_feats, full_train_y, train_size = 0.8)

In [16]:
# sm = SMOTE(random_state=12, ratio = 1.0)
# train_X_res, train_y_res = sm.fit_sample(train_X, train_y)
#train_X_res = pd.DataFrame(train_X_res, columns=train_X.columns)

In [ ]:
### reducing dimension using PCA

In [35]:
pca = PCA(n_components=20)
pca.fit(train_X.drop("CODE_GENDER_XNA", axis=1))
train_X_pca = pca.transform(train_X.drop("CODE_GENDER_XNA", axis=1))
valid_X_pca = pca.transform(valid_X.drop("CODE_GENDER_XNA", axis=1))
test_feats_pca = pca.transform(test_feats)

In [36]:
# ### RF classifier
# params_rf={
#     'max_depth': [20, 40, 60], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
#     'n_estimators': [100, 300, 500], #[1000,2000,3000]
# }

# rf_clf = RandomForestClassifier()
# rf = GridSearchCV(rf_clf,
#                   params_rf,
#                   cv=3,
#                   scoring="roc_auc",
#                   n_jobs=1,
#                   verbose=2)
# rf.fit(train_X_res.drop("CODE_GENDER_XNA", axis=1), train_y_res)
# best_est_rf = rf.best_estimator_
# print(best_est_rf)
# valid_probs_rf = rf.predict_proba(valid_X)[:,1]
# valid_preds_rf = rf.predict(valid_X)

In [37]:
# trainrf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=40, max_features='auto', max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=1, min_samples_split=2,
#             min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
#             oob_score=False, random_state=None, verbose=0,
#             warm_start=False)

# trainrf.fit(train_X.drop("CODE_GENDER_XNA", axis=1), train_y)
# valid_probs_rf = trainrf.predict_proba(valid_X.drop("CODE_GENDER_XNA", axis=1))[:,1]
# valid_preds_rf = trainrf.predict(valid_X.drop("CODE_GENDER_XNA", axis=1))

In [38]:
# print(accuracy_score(valid_y, valid_preds_rf))
# print(roc_auc_score(valid_y, valid_probs_rf))

In [39]:
params={
    'max_depth': [3, 5], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
    'subsample': [0.6, 0.8], #[0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.5, 0.7], #[0.5,0.6,0.7,0.8],
    'n_estimators': [300, 500], #[1000,2000,3000]
    'reg_alpha': [0.01, 0.05] #[0.01, 0.02, 0.03, 0.04]
}

xgb_clf = xgb.XGBClassifier(missing=9999999999)
rs = GridSearchCV(xgb_clf,
                  params,
                  cv=3,
                  scoring="roc_auc",
                  n_jobs=1,
                  verbose=2)
rs.fit(train_X_pca, train_y)
best_est = rs.best_estimator_
print(best_est)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] subsample=0.6, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300 
[CV]  subsample=0.6, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300, total=   3.2s
[CV] subsample=0.6, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV]  subsample=0.6, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300, total=   3.3s
[CV] subsample=0.6, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300 
[CV]  subsample=0.6, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300, total=   5.7s
[CV] subsample=0.8, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300 
[CV]  subsample=0.8, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300, total=   3.8s
[CV] subsample=0.8, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300 
[CV]  subsample=0.8, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300, total=   3.9s
[CV] subsample=0.8, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300 
[CV]  subsample=0.8, reg_alpha=0.01, colsample_bytree=0.5, max_depth=3, n_estimators=300, total=   3.7s
[CV] subsample=0.6, reg_alpha=0.05, colsample_bytree=0.5, max_depth=3, n_estimators=300 
[CV]  subsample=0.6, reg_alpha=0.05

[CV]  subsample=0.6, reg_alpha=0.05, colsample_bytree=0.5, max_depth=5, n_estimators=500, total=  12.4s
[CV] subsample=0.8, reg_alpha=0.05, colsample_bytree=0.5, max_depth=5, n_estimators=500 
[CV]  subsample=0.8, reg_alpha=0.05, colsample_bytree=0.5, max_depth=5, n_estimators=500, total=  13.6s
[CV] subsample=0.8, reg_alpha=0.05, colsample_bytree=0.5, max_depth=5, n_estimators=500 
[CV]  subsample=0.8, reg_alpha=0.05, colsample_bytree=0.5, max_depth=5, n_estimators=500, total=  13.6s
[CV] subsample=0.8, reg_alpha=0.05, colsample_bytree=0.5, max_depth=5, n_estimators=500 
[CV]  subsample=0.8, reg_alpha=0.05, colsample_bytree=0.5, max_depth=5, n_estimators=500, total=  13.5s
[CV] subsample=0.6, reg_alpha=0.01, colsample_bytree=0.7, max_depth=3, n_estimators=300 
[CV]  subsample=0.6, reg_alpha=0.01, colsample_bytree=0.7, max_depth=3, n_estimators=300, total=   5.2s
[CV] subsample=0.6, reg_alpha=0.01, colsample_bytree=0.7, max_depth=3, n_estimators=300 
[CV]  subsample=0.6, reg_alpha=0.01

[CV]  subsample=0.8, reg_alpha=0.01, colsample_bytree=0.7, max_depth=5, n_estimators=500, total=  16.3s
[CV] subsample=0.8, reg_alpha=0.01, colsample_bytree=0.7, max_depth=5, n_estimators=500 
[CV]  subsample=0.8, reg_alpha=0.01, colsample_bytree=0.7, max_depth=5, n_estimators=500, total=  14.8s
[CV] subsample=0.8, reg_alpha=0.01, colsample_bytree=0.7, max_depth=5, n_estimators=500 
[CV]  subsample=0.8, reg_alpha=0.01, colsample_bytree=0.7, max_depth=5, n_estimators=500, total=  15.5s
[CV] subsample=0.6, reg_alpha=0.05, colsample_bytree=0.7, max_depth=5, n_estimators=500 
[CV]  subsample=0.6, reg_alpha=0.05, colsample_bytree=0.7, max_depth=5, n_estimators=500, total=  13.3s
[CV] subsample=0.6, reg_alpha=0.05, colsample_bytree=0.7, max_depth=5, n_estimators=500 
[CV]  subsample=0.6, reg_alpha=0.05, colsample_bytree=0.7, max_depth=5, n_estimators=500, total=  14.4s
[CV] subsample=0.6, reg_alpha=0.05, colsample_bytree=0.7, max_depth=5, n_estimators=500 
[CV]  subsample=0.6, reg_alpha=0.05

[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed: 14.7min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=9999999999, n_estimators=300,
       nthread=-1, objective='binary:logistic', reg_alpha=0.01,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.8)


In [40]:
# xgb_single = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
#        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
#        min_child_weight=1, missing=9999999999, n_estimators=300,
#        nthread=-1, objective='binary:logistic', reg_alpha=0.05,
#        reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
#        subsample=0.8)

# xgb_single.fit(train_X, train_y)
# valid_probs_xgb_single = xgb_single.predict_proba(valid_X)[:,1]
# valid_preds_xgb_single = xgb_single.predict(valid_X)
# print(accuracy_score(valid_y, valid_preds_xgb_single))
# print(roc_auc_score(valid_y, valid_probs_xgb_single))

In [42]:
valid_probs_rs = rs.predict_proba(valid_X_pca)[:,1]
valid_preds_rs= rs.predict(valid_X_pca)
print(accuracy_score(valid_y, valid_preds_rs))
print(roc_auc_score(valid_y, valid_probs_rs))

0.92172009785
0.66251741459


In [46]:
### Prepare submission file and save to disk
result_df = pd.DataFrame({'SK_ID_CURR':test.SK_ID_CURR.values, "TARGET":rs.predict_proba(test_feats)[:,1]})
result_df.to_csv("submimssion_3_xgboost_gridsearch_allfeatures_smote.csv", index=False)

In [47]:
full_train.columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE',
       'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE',
       'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
       'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_2',
       'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_6',
       'FLAG_DOCUMENT_8'],
      dtype='object')

In [53]:
full_train["FLAG_OWN_CAR"].unique()

AttributeError: 'numpy.ndarray' object has no attribute 'values'